zi2zi: Master Chinese Calligraphy with Conditional Adversarial Networks


### 連接 google drive 並確認 workspace.

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

workspace = '110-2/字體設計與文字編碼/HW/ct2022s/hw05'

# Try to aceess your workspace, or create it.
try:
  os.chdir(os.path.join('/content/drive/MyDrive/', workspace))
except:
  os.mkdir(os.path.join('/content/drive/MyDrive/', workspace))
  os.chdir(os.path.join('/content/drive/MyDrive/', workspace))

Mounted at /content/drive


### 確認環境並下載 HW07📡

In [2]:
%tensorflow_version 1.x
!pip install scipy==1.2.1
!git clone https://github.com/t110598009/zi2zi.git
# Acess zi2zi
os.chdir('zi2zi')

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.8 MB 1.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.21.6 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


### Preprocess

To avoid IO bottleneck, preprocessing is necessary to pickle your data into binary and persist in memory during training.

First run the below command to get the font images:  
```
python font2img.py --src_font=font/'source font' --dst_font=font/'Target font' --charset=CN_T  --sample_count=1000 --sample_dir='Your sample dir' --label=0  --filter=1 --shuffle=1
```

#### 建立欲訓練或驗證的圖片

In [5]:
# 建立 sample 資料夾
!mkdir image_train
!mkdir image_val

mkdir: cannot create directory ‘image_train’: File exists


In [6]:
!python font2img.py --src_font=font/NotoSansTC-Black.otf --dst_font=font/BoutiqueBitmap7x7.ttf --charset=TWTrain  --sample_count=1000 --sample_dir=image_train --label=1 --filter=1 --shuffle=1
!python font2img.py --src_font=font/NotoSansTC-Black.otf --dst_font=font/BoutiqueBitmap7x7.ttf --charset=TWVal  --sample_count=670 --sample_dir=image_val --label=1 --filter=1 --shuffle=0


filter hashes -> 
processed 100 chars
processed 200 chars
processed 300 chars
processed 400 chars
processed 500 chars
processed 600 chars
processed 700 chars
processed 800 chars
processed 900 chars
processed 1000 chars
filter hashes -> -8539358471795229346
processed 100 chars
processed 200 chars
processed 300 chars
processed 400 chars
processed 500 chars
processed 600 chars


#### 建立訓練、驗證資料 object
After obtaining all images, run package.py to pickle the images and their corresponding labels into binary format:

跑完後會得到 train.obj and val.obj 在 save_dir 資料夾。

In [7]:
!python package.py --dir=image_train --save_dir=experiment/data --split_ratio=0.1

img image_train/1_u+4e10.jpg 1
img image_train/1_u+4e15.jpg 1
img image_train/1_u+4e4d.jpg 1
img image_train/1_u+4e4e.jpg 1
img image_train/1_u+4e53.jpg 1
img image_train/1_u+4e5e.jpg 1
img image_train/1_u+4e5f.jpg 1
img image_train/1_u+4e69.jpg 1
img image_train/1_u+4e82.jpg 1
img image_train/1_u+4e8b.jpg 1
img image_train/1_u+4e92.jpg 1
img image_train/1_u+4e9b.jpg 1
img image_train/1_u+4e9f.jpg 1
img image_train/1_u+4ec4.jpg 1
img image_train/1_u+4ed6.jpg 1
img image_train/1_u+4ed7.jpg 1
img image_train/1_u+4ede.jpg 1
img image_train/1_u+4f30.jpg 1
img image_train/1_u+4f3d.jpg 1
img image_train/1_u+4f43.jpg 1
img image_train/1_u+4f5e.jpg 1
img image_train/1_u+4f88.jpg 1
img image_train/1_u+4fae.jpg 1
img image_train/1_u+4fd1.jpg 1
img image_train/1_u+4fde.jpg 1
img image_train/1_u+4ff1.jpg 1
img image_train/1_u+5000.jpg 1
img image_train/1_u+5011.jpg 1
img image_train/1_u+5012.jpg 1
img image_train/1_u+5016.jpg 1
img image_train/1_u+5028.jpg 1
img image_train/1_u+503c.jpg 1
img imag

In [8]:
!python package.py --dir=image_val --save_dir=experiment/data/val --split_ratio=1

img image_val/1_u+3002.jpg 1
img image_val/1_u+4e00.jpg 1
img image_val/1_u+4e03.jpg 1
img image_val/1_u+4e09.jpg 1
img image_val/1_u+4e0a.jpg 1
img image_val/1_u+4e0b.jpg 1
img image_val/1_u+4e0d.jpg 1
img image_val/1_u+4e16.jpg 1
img image_val/1_u+4e1e.jpg 1
img image_val/1_u+4e2d.jpg 1
img image_val/1_u+4e3b.jpg 1
img image_val/1_u+4e43.jpg 1
img image_val/1_u+4e45.jpg 1
img image_val/1_u+4e4b.jpg 1
img image_val/1_u+4e58.jpg 1
img image_val/1_u+4e5d.jpg 1
img image_val/1_u+4e91.jpg 1
img image_val/1_u+4e94.jpg 1
img image_val/1_u+4eba.jpg 1
img image_val/1_u+4ed9.jpg 1
img image_val/1_u+4ee4.jpg 1
img image_val/1_u+4ee5.jpg 1
img image_val/1_u+4ef2.jpg 1
img image_val/1_u+4efb.jpg 1
img image_val/1_u+4f2f.jpg 1
img image_val/1_u+4f3c.jpg 1
img image_val/1_u+4f46.jpg 1
img image_val/1_u+4f51.jpg 1
img image_val/1_u+4f55.jpg 1
img image_val/1_u+4f59.jpg 1
img image_val/1_u+4f5c.jpg 1
img image_val/1_u+4f69.jpg 1
img image_val/1_u+4f73.jpg 1
img image_val/1_u+4f7f.jpg 1
img image_val/

### 訓練模型

In [13]:
!python train.py --experiment_dir=experiment --experiment_id=2 --batch_size=32 --lr=0.001 --epoch=40 --sample_steps=50 --schedule=20 --L1_penalty=100 --Lconst_penalty=15



W0528 13:52:16.268052 139906067920768 module_wrapper.py:139] From train.py:46: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W0528 13:52:16.268325 139906067920768 module_wrapper.py:139] From train.py:49: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2022-05-28 13:52:16.274043: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2022-05-28 13:52:16.274252: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x13832c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-28 13:52:16.274285: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-05-28 13:52:16.276096: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-28 13:52:16.500506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA 

### Infer and Interpolate

In [15]:
!python infer.py --model_dir=experiment/checkpoint/experiment_2_batch_32 --batch_size=1 --source_obj=experiment/data/val/val.obj --embedding_ids=1 --save_dir=experiment/infer_4



W0528 15:29:11.977475 140262176327552 module_wrapper.py:139] From infer.py:37: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.


W0528 15:29:11.977770 140262176327552 module_wrapper.py:139] From infer.py:38: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W0528 15:29:11.977970 140262176327552 module_wrapper.py:139] From infer.py:41: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2022-05-28 15:29:11.983835: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2022-05-28 15:29:11.984068: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x17fd2c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-28 15:29:11.984103: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-05-28 15:29:11.985918: I tensorflow/stream_executor/platform/default/dso_load

#Acknowledgements
原作者:
* [zi2zi](https://github.com/kaonashi-tyc/zi2zi) by kaonashi-tyc(original author)

Code derived and rehashed from:

* [pix2pix-tensorflow](https://github.com/yenchenlin/pix2pix-tensorflow) by yenchenlin
* [Domain Transfer Network](https://github.com/yunjey/domain-transfer-network) by yunjey
* [ac-gan](https://github.com/buriburisuri/ac-gan) by buriburisuri
* [dc-gan](https://github.com/carpedm20/DCGAN-tensorflow) by carpedm20
* [origianl pix2pix torch code](https://github.com/phillipi/pix2pix) by phillipi